In [1]:
!pip install ultralytics
!pip install requests
!pip install keyboard


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import torch
import numpy as np
import cv2
from ultralytics import YOLO

import threading
import requests
import subprocess
import keyboard
import datetime

Bot Link : https://t.me/Alert_qbot

To obtain user chatID :
https://api.telegram.org/bot7031174156:AAE9_px2LX0qpsrg5zTOxWDVkB2Yiexc-9g/getupdates

In [23]:
class AnomalyDetection:
    def __init__(self, capture_source, camera_name):
        self.capture_source = capture_source
        self.camera_name = camera_name
        self.msg_sent = False
        self.frames=[]
        self.no_detct = 0
        i=-1
        # Device info
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        # Model info
        self.model = YOLO(r"C:\Users\rayan\OneDrive\Documents\Argus\model.pt")

    def send_msg(self, msg):
        print(msg)

    def frames_to_video(self, out_path, fps=2):
    # Determine the shape of the frames
        size = (self.frames[0].shape[1], self.frames[0].shape[0])
        # Define the codec and create VideoWriter object
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Choose the codec (here, MP4V)
        out = cv2.VideoWriter(out_path, fourcc, fps, size)

        # Write each frame to the video
        for frame in self.frames:
            out.write(frame)

        # Release the VideoWriter object
        out.release()
        print(f"Video saved to: {out_path}")


    def predict(self, im0):
        clss = []
        class_labels = {0: "Accident", 1: "Fight", 2: "Fire", 3: "Smoke"}
        results = self.model.predict(source=im0, conf=0.6)

        for r in results:
            clss = r.boxes.cls.cpu().numpy().astype(int)
            if len(clss) > 0:
                self.no_detct = 0
                self.frames.append(im0)
                if not self.msg_sent:
                    for cl in clss:
                        self.send_msg(class_labels[cl])
                    self.msg_sent = True
            else:
                self.no_detct += 1
                self.msg_sent = False

        if self.no_detct > 10 and len(self.frames) > 0:
            self.no_detct = 0
            current_datetime = datetime.datetime.now().strftime(r"%Y-%m-%d\%H-%M-%S")
            out_path = "C:/Users/rayan/OneDrive/Documents/Argus/please.txt"
            with open(out_path, "a") as f:
                f.write(f"Camera 1 {current_datetime} {','.join(class_labels[cl] for cl in clss) or 'No anomaly detected'}\n")

    def call(self):
        cap = cv2.VideoCapture(self.capture_source)
        assert cap.isOpened()
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 640)
        while True:
            ret, im0 = cap.read()
            assert ret
            self.predict(im0)
            #cv2.imshow(f'{self.camera_name} - Anomaly Detection', im0)
            if cv2.waitKey(5) & 0xFF == 27 or keyboard.is_pressed('esc'):
                break
        cap.release()
        cv2.destroyAllWindows()

# Instantiate anomaly detection for each camera
detector1 = AnomalyDetection(capture_source=0, camera_name='Camera 1')
#detector2 = AnomalyDetection(capture_source='http://192.168.63.251:8080/video', camera_name='Camera 2')

# Start anomaly detection for each camera in separate threads
thread1 = threading.Thread(target=detector1.call)
#thread2 = threading.Thread(target=detector2.call)

# Start both threads
thread1.start()
#thread2.start()

# Wait for both threads to finish
thread1.join()
#thread2.join()


0: 480x640 (no detections), 422.4ms
Speed: 2.0ms preprocess, 422.4ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 419.1ms
Speed: 2.0ms preprocess, 419.1ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 395.8ms
Speed: 1.0ms preprocess, 395.8ms inference, 10.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 403.4ms
Speed: 1.6ms preprocess, 403.4ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 435.8ms
Speed: 2.0ms preprocess, 435.8ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 459.8ms
Speed: 2.0ms preprocess, 459.8ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 428.7ms
Speed: 2.0ms preprocess, 428.7ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 440.7ms
Speed: 2.0ms prep

In [3]:
class AnomalyDetection:
    def __init__(self, capture_source, camera_name):
        self.capture_source = capture_source
        self.camera_name = camera_name
        self.msg_sent = False
        # Device info
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        # Model info
        self.model = YOLO('best.pt')  
    
    def send_msg(self, msg):
        subprocess.call(['cscript', 'popup.vbs', f"{self.camera_name} - Anomaly Detected: {msg}"], shell=True)
    
    def predict(self, im0):
        #class_ids = 0
        clss=[]
        results = self.model.predict(source=im0, conf=0.4)
        anomaly = ["Accident", "Fight", "Fire", "Smoke"]
        for r in results:
            clss = r.boxes.cls.cpu().numpy().astype(int)
            #if len(clss) > 0:
                #class_ids += 1
        if len(clss)>0:
            if not self.msg_sent:
                for cl in clss:
                    self.send_msg(anomaly[cl])
                self.msg_sent = True
        else:
            self.msg_sent = False
            
    def __call__(self):
        cap = cv2.VideoCapture(self.capture_source)
        assert cap.isOpened()
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 640)
        while True:
            ret, im0 = cap.read()
            assert ret
            self.predict(im0)
            #cv2.imshow(f'{self.camera_name} - Anomaly Detection', im0)
            if cv2.waitKey(5) & 0xFF == 27 or keyboard.is_pressed('esc'):
                break
        cap.release()
        cv2.destroyAllWindows()

# Instantiate anomaly detection for each camera
detector1 = AnomalyDetection(capture_source='http://192.168.63.52:8080/video', camera_name='Camera 1')
detector2 = AnomalyDetection(capture_source='http://192.168.63.251:8080/video', camera_name='Camera 2')

# Start anomaly detection for each camera in separate threads
thread1 = threading.Thread(target=detector1.__call__)
thread2 = threading.Thread(target=detector2.__call__)

# Start both threads
thread1.start()
thread2.start()

# Wait for both threads to finish
thread1.join()
thread2.join()

FileNotFoundError: [Errno 2] No such file or directory: 'best.pt'

In [ ]:
class AnomalyDetection:
    def __init__(self, capture_source):
        self.capture_source = capture_source
        self.msg_sent = False
        # Device info
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        # Model info
        self.model = YOLO('best.pt') 
        #out = cv2.VideoWriter(r'C:\Users\lenovo\Downloads\Anomaly detection\Detection', cv2.VideoWriter_fourcc(*'XVID'), fps, frame_size) 
    
    def predict(self, im0):
        #class_ids = 0
        results = self.model.predict(source=im0, conf=0.2,save=True)
        anomaly = ["Accident", "Fight", "Fire", "Smoke"]
        for r in results:
            clss = r.boxes.cls.cpu().numpy().astype(int)
            #out.write(im0)
        if len(clss)>0:
            if not self.msg_sent:
                for cl in clss:
                    print(cl)
                self.msg_sent = True
        else:
            self.msg_sent = False
            
    def __call__(self):
        cap = cv2.VideoCapture(self.capture_source)
        assert cap.isOpened()
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 640)
        # # Obtain frames per second (fps)
        # fps = cap.get(cv2.CAP_PROP_FPS)
        # # Obtain frame size
        # frame_size = (cv2.CAP_PROP_FRAME_WIDTH,cv2.CAP_PROP_FRAME_HEIGHT)
        while True:
            ret, im0 = cap.read()
            assert ret
            self.predict(im0)
            cv2.imshow('Anomaly Detection', im0)
            if cv2.waitKey(5) & 0xFF == 27 or keyboard.is_pressed('esc'):
                break
        cap.release()
        cv2.destroyAllWindows()


detector = AnomalyDetection(capture_source=0)
detector()

In [ ]:


##################################################################################################################################################################################################################################

class ObjectDetection:
    def __init__(self):
        # default parameters
       # self.capture_index = capture_index
        self.msg_sent = False
        count=1
        # model information
        self.model = self.load_model()
        # device information
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        
    def load_model(self):
        model=YOLO(r"C:\Users\lenovo\Downloads\Anomaly detection\best.pt")
        model.fuse()
        return model    

    def predict(self, im0):
        results = self.model(im0)
        return results

    def plot_bboxes(self, results, im0):
        class_ids = []
        for r in results:
          clss = r.boxes.cls.cpu().numpy()
        return im0, clss, class_ids
            
    def __call__(self):
        #cap = cv2.VideoCapture(self.capture_index)
        cap=cv2.VideoCapture('http://192.168.63.52:8080/video')
        assert cap.isOpened()
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 640)
        frame_count = 0
        while True:
            self.start_time = time()
            ret, im0 = cap.read()
            assert ret
            frame_count=frame_count+1
            if frame_count==6:
              results = self.predict(im0)
              im0, clss, class_ids = self.plot_bboxes(results, im0)

              # Only send alert If not sent before
              if len(class_ids) > 0:
                  if not self.msg_sent:
                      cv2.imwrite(f"C:/Users/lenovo/Downloads/Anomaly detection/Detection/anomaly_{count}.jpg",im0)
                      count=count+1
                      #send_msg(im0 , clss)
                      self.msg_sent = True
              else:
                  self.msg_sent = False
              frame_count=0

            cv2.imshow('Anomaly Detection', im0)
            if cv2.waitKey(5) & 0xFF == 27:
                break
        cap.release()
        cv2.destroyAllWindows()
        server.quit()

detector = ObjectDetection()
detector()

In [1]:
pip install opencv-python


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\lenovo\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [ ]:
import cv2
from google.colab.patches import cv2_imshow

# Load pre-trained face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Function to detect faces and extract features
def detect_and_extract_features(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    # Extract features from detected faces
    keypoints_list = []
    descriptors_list = []
    faces_coordinates = []
    for (x, y, w, h) in faces:
        # Draw bounding box around the face
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        faces_coordinates.append((x, y, w, h))
        
        # Extract face region
        face_roi = gray[y:y+h, x:x+w]
        
        # Initialize ORB feature descriptor
        orb = cv2.ORB_create()
        
        # Detect keypoints and compute descriptors
        kp, des = orb.detectAndCompute(face_roi, None)
        keypoints_list.append(kp)
        descriptors_list.append(des)
    
    return keypoints_list, descriptors_list, faces_coordinates

# Function to match features between two sets of descriptors
def match_features(descriptors1, descriptors2):
    if descriptors1 is None or descriptors2 is None:
        return []
    
    # Initialize brute force matcher
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    
    # Match descriptors
    matches = bf.match(descriptors1, descriptors2)
    
    # Sort matches based on distance
    matches = sorted(matches, key=lambda x: x.distance)
    
    return matches

# Function to display matched faces
def display_matched_faces(frame1, frame2, keypoints1, keypoints2, matches, faces1, faces2):
    # Draw bounding boxes around detected faces
    for (x, y, w, h) in faces1:
        cv2.rectangle(frame1, (x, y), (x+w, y+h), (255, 0, 0), 2)
    for (x, y, w, h) in faces2:
        cv2.rectangle(frame2, (x, y), (x+w, y+h), (255, 0, 0), 2)
    
    # Draw matches on the frames
    matched_image = cv2.drawMatches(frame1, keypoints1, frame2, keypoints2, matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    
    # Display matched faces
    cv2_imshow(matched_image)
    cv2.waitKey(0)

if __name__ == "__main__":
    # Path to the videos
    video_path1 = '/content/1.mp4'
    video_path2 = '/content/2.mp4'

    # Open the videos
    cap1 = cv2.VideoCapture(video_path1)
    cap2 = cv2.VideoCapture(video_path2)

    # Read the frames of each video
    frame1_list = []
    frame2_list = []
    while True:
        ret1, frame1 = cap1.read()
        ret2, frame2 = cap2.read()
        
        if not ret1 or not ret2:
            break
        
        frame1_list.append(frame1)
        frame2_list.append(frame2)

    # Detect and extract features from the frames of each video
    keypoints1_list, descriptors1_list, faces1_list = [], [], []
    keypoints2_list, descriptors2_list, faces2_list = [], [], []
    for frame1, frame2 in zip(frame1_list, frame2_list):
        keypoints1, descriptors1, faces1 = detect_and_extract_features(frame1)
        keypoints2, descriptors2, faces2 = detect_and_extract_features(frame2)
        
        keypoints1_list.append(keypoints1)
        descriptors1_list.append(descriptors1)
        faces1_list.append(faces1)
        
        keypoints2_list.append(keypoints2)
        descriptors2_list.append(descriptors2)
        faces2_list.append(faces2)

    # Match features between the frames of the two videos
    matches_list = []
    for descriptors1, descriptors2 in zip(descriptors1_list, descriptors2_list):
        matches = match_features(descriptors1[0] if descriptors1 else None, descriptors2[0] if descriptors2 else None)
        matches_list.append(matches)

    # Display matched faces for each pair of frames
    for frame1, frame2, keypoints1, keypoints2, matches, faces1, faces2 in zip(frame1_list, frame2_list, keypoints1_list, keypoints2_list, matches_list, faces1_list, faces2_list):
        display_matched_faces(frame1.copy(), frame2.copy(), keypoints1[0] if keypoints1 else None, keypoints2[0] if keypoints2 else None, matches, faces1, faces2)

    # Release the video capture objects
    cap1.release()
    cap2.release()
